In [16]:
import numpy as np
import copy
from nibabel import trackvis as tv
from dipy.tracking.streamline import Streamlines
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_fnames
import vtk
from dipy.viz import window, actor
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import load_trk, save_trk
from dipy.segment.metric import ResampleFeature, AveragePointwiseEuclideanMetric
from dipy.io.image import load_nifti
from dipy.tracking import utils
from dipy.viz import window, actor
from time import sleep
from dipy.data import two_cingulum_bundles
from dipy.align.streamlinear import StreamlineLinearRegistration
from dipy.tracking.streamline import set_number_of_points

In [17]:
def show_both_bundles(bundles, colors=None, show=True, fname=None):

    ren = window.Renderer()
    ren.SetBackground(1., 1, 1)
    for (i, bundle) in enumerate(bundles):
        color = colors[i]
        lines_actor = actor.streamtube(bundle, color, linewidth=0.05)
        #lines_actor.RotateX(-90)
        #lines_actor.RotateZ(90)
        ren.add(lines_actor)
    if show:
        window.show(ren)
    if fname is not None:
        sleep(1)
        window.record(ren, n_frames=1, out_path=fname, size=(900, 900))

In [30]:
#indicate the ROIs interested, note target_l>target_r
target_l = 51
target_r = 1051

In [31]:
#general information
l = ['N54717','N54718','N54719','N54720','N54722','N54759','N54760','N54761','N54762','N54763','N54764','N54765','N54766','N54770','N54771','N54772','N54798','N54801','N54802','N54803','N54804','N54805','N54806','N54807','N54818','N54824','N54825','N54826','N54837','N54838','N54843','N54844','N54856','N54857','N54858','N54859','N54860','N54861','N54873','N54874','N54875','N54876','N54877','N54879','N54880','N54891','N54892','N54893','N54897','N54898','N54899','N54900','N54915','N54916','N54917']
gen4idx = [1,2,3,4,7,8,9,10,12,13,52,53,54]
gen3idx = [14,15,16,17,18,19,30,21,22,23]
gen4selectidx = [2, 3, 7, 8, 9, 10, 13, 52, 53, 54]
gen4 = [l[i] for i in gen4selectidx]
gen3 = [l[j] for j in gen3idx]
gen = [gen3, gen4]

#exclude N54900
# gen0idx = [5,6,11,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
# gen0 = [l[k] for k in gen0idx]
# # oldidx = [5,6,11,24,25,26,27,28,29,30,31,32,33,34,45,46,47,48,49,50]
# # old = [l[o] for o in oldidx]
# oldselectidx = [6, 25, 27, 28, 29, 32, 33, 45, 46, 50]
# old = [l[o] for o in oldselectidx]
# youngidx = [35,36,37,38,39,40,41,42,43,44]
# young = [l[y] for y in youngidx]
# age = [young,old]


In [32]:
# import random
# random.seed(123)
# oldidx = random.sample(oldidx,10)
# old = [l[o] for o in oldidx]
# age = [young,old]

In [33]:
#set path
mypath = '/Users/alex/code/Wenlin/data'
outpath = '/Users/alex/code/Wenlin/Tracts_Registration/results'

In [34]:
#set parameter
num_points1 = 100
distance1 = 1
feature1 = ResampleFeature(nb_points=num_points1)
metric1 = AveragePointwiseEuclideanMetric(feature=feature1)

#group cluster parameter
num_points2 = 100
distance2 = 1
feature2 = ResampleFeature(nb_points=num_points2)
metric2 = AveragePointwiseEuclideanMetric(feature=feature2) 

In [35]:
#load the control animal
streams_control,hdr_control = load_trk(mypath+'/wenlin_results/N54900_bmCSA_detr_small.trk')
labels_control, affine_labels_control = load_nifti(mypath+'/wenlin_data/labels/fa_labels_warp_N54900_RAS.nii.gz') 

In [36]:
#uncomment this cell to generate both left and right
# labels_pair_control = copy.copy(labels_control)
# nonz_control = np.nonzero(labels_pair_control)
# for i in range(len(nonz_control[0])):
#     if labels_pair_control[nonz_control[0][i], nonz_control[1][i], nonz_control[2][i]]>=1000:
#         labels_pair_control[nonz_control[0][i], nonz_control[1][i], nonz_control[2][i]] -= 1000
# print('pair labels generated')

# #pair labels target control animals
# streams_fix_control = lambda : (sl for sl in streams_control if len(sl)>1)
# streamlines_control = Streamlines(streams_fix_control())
# M_control, grouping_control = utils.connectivity_matrix(streamlines_control, labels_pair_control, 
#                                                         affine=affine_labels_control, return_mapping=True,
#                                                         mapping_as_streamlines=True)

# target_streamlines_control = grouping_control[target_l, target_r]

In [37]:
#target control animals
streams_fix_control = lambda : (sl for sl in streams_control if len(sl)>1)
streamlines_control = Streamlines(streams_fix_control())
M_control, grouping_control = utils.connectivity_matrix(streamlines_control, labels_control, 
                                                        affine=affine_labels_control, return_mapping=True,
                                                        mapping_as_streamlines=True)

target_streamlines_control = grouping_control[target_l, target_r]

In [38]:
#cluster control animals
target_qb_control = QuickBundles(threshold=distance1,metric=metric1)
target_clusters_control = target_qb_control.cluster(target_streamlines_control)
print("Control Nb. clusters:", len(target_clusters_control))

Control Nb. clusters: 117


In [39]:
#group calculation
for k in range(2):
    gengroup = gen[k]
    createVar = locals()
    createVar['groupclusters'+str(k+3)] = [] #create empty list, 4-old, 3-young
    animallist = []
    print('Group'+str(k+3)+' started')
    for j in range(len(gengroup)):
        runno = gengroup[j]
        animallist.append(runno)
        streams,hdr = load_trk(mypath+'/wenlin_results/'+runno+'_bmCSA_detr_small.trk')
        labels, affine_labels = load_nifti(mypath+'/wenlin_data/labels/fa_labels_warp_'+runno+'_RAS.nii.gz') 

#         labels_pair = copy.copy(labels)
#         nonz = np.nonzero(labels_pair)
#         for i in range(len(nonz[0])):
#             if labels_pair[nonz[0][i], nonz[1][i], nonz[2][i]]>=1000:
#                 labels_pair[nonz[0][i], nonz[1][i], nonz[2][i]] -= 1000
#         #print('pair labels generated')
        
        #target moving animals
        streams_fix = lambda : (sl for sl in streams if len(sl)>1)
        streamlines = Streamlines(streams_fix())
        M, grouping = utils.connectivity_matrix(streamlines, labels, affine=affine_labels, 
                                                return_mapping=True,mapping_as_streamlines=True)

        target_streamlines = grouping[target_l, target_r]
        
        
        target_qb = QuickBundles(threshold=distance1,metric=metric1)
        target_clusters = target_qb.cluster(target_streamlines)
        print('NO.'+str(j+1)+' '+runno+" Nb. clusters:", len(target_clusters))
        #registration
        srr = StreamlineLinearRegistration()
        srm = srr.optimize(static=target_clusters_control.centroids, moving=target_clusters.centroids)
        target_subj_aligned = srm.transform(target_clusters.centroids)
        
        locals()['groupclusters'+str(k+3)].extend(target_subj_aligned)
    print('agetype-'+str(k+3)+' finished. '+'total number of clusters for group'+ str(k+3) 
          + ': {}'.format(len(locals()['groupclusters'+str(k+3)])))
    print('animal list: ', animallist)
    print(' ')
        
        

Group3 started
NO.1 N54771 Nb. clusters: 84
NO.2 N54772 Nb. clusters: 83
NO.3 N54798 Nb. clusters: 73
NO.4 N54801 Nb. clusters: 81
NO.5 N54802 Nb. clusters: 133
NO.6 N54803 Nb. clusters: 93
NO.7 N54843 Nb. clusters: 81
NO.8 N54805 Nb. clusters: 85
NO.9 N54806 Nb. clusters: 63
NO.10 N54807 Nb. clusters: 91
agetype-3 finished. total number of clusters for group3: 867
animal list:  ['N54771', 'N54772', 'N54798', 'N54801', 'N54802', 'N54803', 'N54843', 'N54805', 'N54806', 'N54807']
 
Group4 started
NO.1 N54719 Nb. clusters: 73
NO.2 N54720 Nb. clusters: 121
NO.3 N54761 Nb. clusters: 76
NO.4 N54762 Nb. clusters: 64
NO.5 N54763 Nb. clusters: 75
NO.6 N54764 Nb. clusters: 72
NO.7 N54770 Nb. clusters: 95
NO.8 N54915 Nb. clusters: 106
NO.9 N54916 Nb. clusters: 98
NO.10 N54917 Nb. clusters: 100
agetype-4 finished. total number of clusters for group4: 880
animal list:  ['N54719', 'N54720', 'N54761', 'N54762', 'N54763', 'N54764', 'N54770', 'N54915', 'N54916', 'N54917']
 


In [40]:
group3_qb = QuickBundles(threshold=distance2,metric=metric2)
group3_clusters = group3_qb.cluster(groupclusters3)
group4_qb = QuickBundles(threshold=distance2,metric=metric2)
group4_clusters = group4_qb.cluster(groupclusters4)

In [41]:
from dipy.io.streamline import save_trk
save_trk('results/'+str(target_l)+'--'+str(target_r)+'11gen3CSA.trk', group3_clusters.centroids, affine=np.eye(4),
         shape=labels.shape)
save_trk('results/'+str(target_l)+'--'+str(target_r)+'11gen4CSA.trk', group4_clusters.centroids, affine=np.eye(4),
         shape=labels.shape)